In [3]:
import SimpleITK as sitk
from supporters import *
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L

In [7]:
path = "D:\Documents\GitHub\Triple-View-R-Net\data\BTCV\BTCV_training\DET0004601_avg.nii.gz"
image_raw = sitk.ReadImage(path)
image = sitk.GetArrayFromImage(image_raw)

print(image.shape)

(14, 256, 256)


In [16]:
label_path = "D:\Documents\GitHub\Triple-View-R-Net\data\BTCV\BTCV_labels\DET0004601_avg_seg.nii.gz"
label_raw = sitk.ReadImage(label_path)
label = sitk.GetArrayFromImage(label_raw)

print(label.shape)

(14, 256, 256)


In [ ]:
explore_3D_array_comparison(image, label)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10,10))

for index, ax in enumerate(axes.flat):
    ax.imshow(image[index], cmap='gray')
    ax.axis('off')
    ax.set_title(f'Slice {index+1}')

plt.suptitle("Display data")
plt.tight_layout()
plt.show()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

input_shape = (2,3,4)
x1 = tf.random.normal(input_shape)
print(x1)
x2 = tf.random.normal(input_shape)
print(x2)

y = L.Add()([x1,x2])
print(y)

print(x1[0][0][0] + x2[0][0][0], y[0][0][0])